In [8]:
#import dependancies
import pandas as pd
import numpy as np
from flask import Flask, render_template, redirect
from pprint import pprint

full_df = pd.read_csv('desktop/p_2_data/district_data_full_clean.csv')

full_df.head()

,NAME,TOTALREV,TFEDREV,TSTREV,TLOCREV,tot_rev_per_s,fed_rev_per_s,st_rev_per_s,loc_rev_per_s,ENROLL,...,ave_school_enrol,ave_prcnt_wht,ave_prcnt_bk,ave_prcnt_hisp,ave_prcnt_asn,ave_prcnt_othr,free_lunch_eligible,not_eligible,total_count,percent_student_body_eligible_for_frl
0,Adams Township School District,4927,268,3442,1217,10.876380,0.591611,7.598234,2.686534,453,...,223.0,98.0,0.5,0.5,0.0,1.0,87.0,106.0,228.0,53.5
1,Addison Community Schools,10189,1130,4035,5024,11.916959,1.321637,4.719298,5.876023,855,...,220.5,88.0,1.0,6.5,0.5,4.5,98.5,108.5,223.5,51.5
2,"Adrian, School District of the City of",37526,6400,21662,9464,12.664867,2.159973,7.310834,3.194060,2963,...,431.6,58.2,4.0,29.4,1.2,7.4,257.4,156.0,445.4,70.4
3,Airport Community Schools,27360,1257,18938,7165,11.072440,0.508701,7.664104,2.899636,2471,...,246.8,91.8,3.2,3.8,0.4,0.6,125.8,122.4,261.8,56.2
4,Akron-Fairgrove Schools,3993,296,1587,2110,14.159574,1.049645,5.627660,7.482270,282,...,143.5,88.5,0.5,7.5,0.5,3.0,93.0,38.0,145.5,74.0


In [3]:
#full df, prune some rows that probably aren't necessary

prune_df = full_df.drop(columns = ['TOTALREV', 'TFEDREV', 'TSTREV', 'TLOCREV', 'TCURINST', 'TOTALEXP'])

prune_df.head()

,NAME,tot_rev_per_s,fed_rev_per_s,st_rev_per_s,loc_rev_per_s,ENROLL,tot_spend_per_s,tot_inst_per_s,lat,lng,ave_school_enrol,ave_prcnt_wht,ave_prcnt_bk,ave_prcnt_hisp,ave_prcnt_asn,ave_prcnt_othr,free_lunch_eligible,not_eligible,total_count,percent_student_body_eligible_for_frl
0,Adams Township School District,10.876380,0.591611,7.598234,2.686534,453,10.154525,5.121413,47.055234,-88.658298,223.0,98.0,0.5,0.5,0.0,1.0,87.0,106.0,228.0,53.5
1,Addison Community Schools,11.916959,1.321637,4.719298,5.876023,855,11.045614,6.376608,41.988254,-84.343413,220.5,88.0,1.0,6.5,0.5,4.5,98.5,108.5,223.5,51.5
2,"Adrian, School District of the City of",12.664867,2.159973,7.310834,3.194060,2963,12.099224,6.657104,41.895706,-84.049822,431.6,58.2,4.0,29.4,1.2,7.4,257.4,156.0,445.4,70.4
3,Airport Community Schools,11.072440,0.508701,7.664104,2.899636,2471,10.782679,5.854310,42.041265,-83.338547,246.8,91.8,3.2,3.8,0.4,0.6,125.8,122.4,261.8,56.2
4,Akron-Fairgrove Schools,14.159574,1.049645,5.627660,7.482270,282,19.244681,6.301418,43.556976,-83.529938,143.5,88.5,0.5,7.5,0.5,3.0,93.0,38.0,145.5,74.0


In [11]:
#use a for loop to package the data up

#break prune_df into single component numpy arrays
district = prune_df['NAME'].to_numpy()
lat = prune_df['lat'].to_numpy()
lng = prune_df['lng'].to_numpy()
tot_rev_per_s = prune_df['tot_rev_per_s'].to_numpy()
fed_rev_per_s = prune_df['fed_rev_per_s'].to_numpy()
st_rev_per_s = prune_df['st_rev_per_s'].to_numpy()
loc_rev_per_s = prune_df['loc_rev_per_s'].to_numpy()
district_enrollment = prune_df['ENROLL'].to_numpy()
tot_spend_per_s = prune_df['tot_spend_per_s'].to_numpy()
tot_inst_per_s = prune_df['tot_inst_per_s'].to_numpy()
ave_school_enrol = prune_df['ave_school_enrol'].to_numpy()
ave_prcnt_wht = prune_df['ave_prcnt_wht'].to_numpy()
ave_prcnt_bk = prune_df['ave_prcnt_bk'].to_numpy()
ave_prcnt_asn = prune_df['ave_prcnt_asn'].to_numpy()
ave_prcnt_othr = prune_df['ave_prcnt_othr'].to_numpy()
prcnt_stdnts_eligible_for_frl  = prune_df['percent_student_body_eligible_for_frl'].to_numpy()


#define an empty list for 
dist_data = []

for d in range(len(district)):
    #initialize the dictionary "entry"
    entry = {}
    
    #add all the things
    entry.update({'district': district[d]})
    entry.update({'lat': lat[d]})
    entry.update({'lng': lng[d]})
    entry.update({'tot_rev_per_s': tot_rev_per_s[d]})
    entry.update({'fed_rev_per_s': fed_rev_per_s[d]})
    entry.update({'st_rev_per_s': st_rev_per_s[d]})
    entry.update({'loc_rev_per_s': loc_rev_per_s[d]})
    entry.update({'district_enrollment': district_enrollment[d]})
    entry.update({'tot_spend_per_s': tot_spend_per_s[d]})
    entry.update({'tot_inst_per_s': tot_inst_per_s[d]})
    entry.update({'ave_school_enrol': ave_school_enrol[d]})
    entry.update({'ave_prcnt_wht': ave_prcnt_wht[d]})
    entry.update({'ave_prcnt_bk': ave_prcnt_bk[d]})
    entry.update({'ave_prcnt_asn': ave_prcnt_asn[d]})
    entry.update({'ave_prcnt_othr': ave_prcnt_othr[d]})
    entry.update({'prcnt_stdnts_eligible_for_frl': prcnt_stdnts_eligible_for_frl[d]})
    
    #append the entry into the dist_data list
    dist_data.append(entry)
    

dist_data_dict = { 'district_data': dist_data}

pprint(dist_data_dict)  

{'district_data': [{'ave_prcnt_asn': 0.0,
                    'ave_prcnt_bk': 0.5,
                    'ave_prcnt_othr': 1.0,
                    'ave_prcnt_wht': 98.0,
                    'ave_school_enrol': 223.0,
                    'district': 'Adams Township School District',
                    'district_enrollment': 453,
                    'fed_rev_per_s': 0.591611479,
                    'lat': 47.05523415,
                    'lng': -88.65829815,
                    'loc_rev_per_s': 2.686534216,
                    'prcnt_stdnts_eligible_for_frl': 53.5,
                    'st_rev_per_s': 7.598233996,
                    'tot_inst_per_s': 5.121412803999999,
                    'tot_rev_per_s': 10.87637969,
                    'tot_spend_per_s': 10.15452539},
                   {'ave_prcnt_asn': 0.5,
                    'ave_prcnt_bk': 1.0,
                    'ave_prcnt_othr': 4.5,
                    'ave_prcnt_wht': 88.0,
                    'ave_school_enrol': 220.5,
     

                   {'ave_prcnt_asn': 1.0,
                    'ave_prcnt_bk': 0.0,
                    'ave_prcnt_othr': 4.0,
                    'ave_prcnt_wht': 92.0,
                    'ave_school_enrol': 543.0,
                    'district': 'Breitung Township School District',
                    'district_enrollment': 1792,
                    'fed_rev_per_s': 0.443080357,
                    'lat': 45.8051943,
                    'lng': -88.1038804,
                    'loc_rev_per_s': 3.841517857,
                    'prcnt_stdnts_eligible_for_frl': 37.0,
                    'st_rev_per_s': 6.554129464,
                    'tot_inst_per_s': 5.997209821,
                    'tot_rev_per_s': 10.83872768,
                    'tot_spend_per_s': 9.747209821},
                   {'ave_prcnt_asn': 0.0,
                    'ave_prcnt_bk': 54.66666667,
                    'ave_prcnt_othr': 5.333333333,
                    'ave_prcnt_wht': 21.66666667,
                    'ave_school_e

                    'lng': -83.96632703,
                    'loc_rev_per_s': 3.608411215,
                    'prcnt_stdnts_eligible_for_frl': 32.33333333,
                    'st_rev_per_s': 7.410280374,
                    'tot_inst_per_s': 6.407476636,
                    'tot_rev_per_s': 11.27850467,
                    'tot_spend_per_s': 10.99065421},
                   {'ave_prcnt_asn': 0.9,
                    'ave_prcnt_bk': 64.5,
                    'ave_prcnt_othr': 5.3,
                    'ave_prcnt_wht': 26.9,
                    'ave_school_enrol': 286.5,
                    'district': 'Clintondale Community Schools',
                    'district_enrollment': 2971,
                    'fed_rev_per_s': 0.846179737,
                    'lat': 42.50668707,
                    'lng': -82.96801612,
                    'loc_rev_per_s': 2.321104005,
                    'prcnt_stdnts_eligible_for_frl': 77.4,
                    'st_rev_per_s': 8.221137664,
                    

                    'district_enrollment': 1694,
                    'fed_rev_per_s': 0.295159386,
                    'lat': 43.60690453,
                    'lng': -83.83071403,
                    'loc_rev_per_s': 3.923848878,
                    'prcnt_stdnts_eligible_for_frl': 38.25,
                    'st_rev_per_s': 6.325265643,
                    'tot_inst_per_s': 5.572609209,
                    'tot_rev_per_s': 10.54427391,
                    'tot_spend_per_s': 12.34533648},
                   {'ave_prcnt_asn': 0.0,
                    'ave_prcnt_bk': 2.0,
                    'ave_prcnt_othr': 0.0,
                    'ave_prcnt_wht': 96.0,
                    'ave_school_enrol': 280.3333333,
                    'district': 'Evart Public Schools',
                    'district_enrollment': 859,
                    'fed_rev_per_s': 1.044237485,
                    'lat': 43.9044708,
                    'lng': -85.26973557,
                    'loc_rev_per_s': 3.314318976,
 

                    'ave_prcnt_wht': 84.0,
                    'ave_school_enrol': 409.25,
                    'district': 'Iron Mountain Public Schools',
                    'district_enrollment': 891,
                    'fed_rev_per_s': 0.271604938,
                    'lat': 45.81967215,
                    'lng': -88.06076462,
                    'loc_rev_per_s': 4.175084175,
                    'prcnt_stdnts_eligible_for_frl': 52.75,
                    'st_rev_per_s': 6.390572391,
                    'tot_inst_per_s': 5.68013468,
                    'tot_rev_per_s': 10.8372615,
                    'tot_spend_per_s': 12.07968575},
                   {'ave_prcnt_asn': 0.0,
                    'ave_prcnt_bk': 1.5,
                    'ave_prcnt_othr': 10.0,
                    'ave_prcnt_wht': 87.0,
                    'ave_school_enrol': 399.0,
                    'district': 'Ironwood Area Schools of Gogebic County',
                    'district_enrollment': 792,
               

                    'ave_prcnt_othr': 1.0,
                    'ave_prcnt_wht': 97.0,
                    'ave_school_enrol': 231.0,
                    'district': 'Mio-AuSable Schools',
                    'district_enrollment': 526,
                    'fed_rev_per_s': 1.161596958,
                    'lat': 44.6531804,
                    'lng': -84.1443072,
                    'loc_rev_per_s': 4.94486692,
                    'prcnt_stdnts_eligible_for_frl': 78.0,
                    'st_rev_per_s': 4.979087452,
                    'tot_inst_per_s': 7.1825095060000015,
                    'tot_rev_per_s': 11.08555133,
                    'tot_spend_per_s': 11.18060837},
                   {'ave_prcnt_asn': 2.0,
                    'ave_prcnt_bk': 10.5,
                    'ave_prcnt_othr': 5.666666667,
                    'ave_prcnt_wht': 79.0,
                    'ave_school_enrol': 630.8333332999998,
                    'district': 'Mona Shores Public School District',
          

                    'st_rev_per_s': 6.9500648510000005,
                    'tot_inst_per_s': 5.771725032000001,
                    'tot_rev_per_s': 10.66731518,
                    'tot_spend_per_s': 9.64461738},
                   {'ave_prcnt_asn': 1.0,
                    'ave_prcnt_bk': 1.333333333,
                    'ave_prcnt_othr': 3.0,
                    'ave_prcnt_wht': 88.0,
                    'ave_school_enrol': 274.3333333,
                    'district': 'Reese Public Schools',
                    'district_enrollment': 822,
                    'fed_rev_per_s': 0.8053527979999999,
                    'lat': 43.45445413,
                    'lng': -83.68357263,
                    'loc_rev_per_s': 3.020681265,
                    'prcnt_stdnts_eligible_for_frl': 51.33333333,
                    'st_rev_per_s': 7.226277372,
                    'tot_inst_per_s': 6.1082725060000005,
                    'tot_rev_per_s': 11.05231144,
                    'tot_spend_per_s': 1

                    'prcnt_stdnts_eligible_for_frl': 58.23529412,
                    'st_rev_per_s': 7.620799731,
                    'tot_inst_per_s': 6.189264112999999,
                    'tot_rev_per_s': 12.25445228,
                    'tot_spend_per_s': 11.62508401},
                   {'ave_prcnt_asn': 1.0,
                    'ave_prcnt_bk': 1.5,
                    'ave_prcnt_othr': 1.5,
                    'ave_prcnt_wht': 93.5,
                    'ave_school_enrol': 187.0,
                    'district': 'Webberville Community Schools',
                    'district_enrollment': 561,
                    'fed_rev_per_s': 0.620320856,
                    'lat': 42.66748205,
                    'lng': -84.17157125,
                    'loc_rev_per_s': 4.409982175,
                    'prcnt_stdnts_eligible_for_frl': 56.5,
                    'st_rev_per_s': 7.445632799,
                    'tot_inst_per_s': 5.495543672,
                    'tot_rev_per_s': 12.47593583,
      